In [3]:
!pip install camel-tools

  Using cached camel_tools-1.5.5-py3-none-any.whl.metadata (10 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.43.4-py3-none-any.whl.metadata (43 kB)
  Using cached emoji-2.12.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached muddler-0.1.3-py3-none-any.whl.metadata (7.5 kB)
  Using cached camel-kenlm-2024.5.6.zip (556 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached camel_tools-1.5.5-py3-none-any.whl (124 kB)
Using cached transformers-4.43.4-py3-none-any.whl (9.4 MB)
Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
Using cached dill-0.3.8-py3-none-any.wh

In [4]:
import os
import csv
import nltk
#nltk.download()
from nltk.tokenize import word_tokenize
import os
import json
import requests
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger

In [5]:
def read_tweets(path):
    tokens = []
    labels = []
    t = []
    l = []
    for token in open(path, encoding='utf-8').read().splitlines(): 
        if token == '':
            tokens.append(t)
            labels.append(l)
            t = []
            l = []
            continue
        splits = token.split()
        t.append(splits[0])
        l.append(splits[1])

    if len(t) > 0 and len(l) > 0:
        tokens.append(t)
        labels.append(l)
        
    return tokens, labels

In [7]:
#tags: https://camel-tools.readthedocs.io/en/latest/reference/camel_morphology_features.html
tag_mapping = {"adj": "ADJ", "adposition": "ADP", "adverb": "ADV", "auxiliary": "AUX", 
               "coordinating conjunction": "CCONJ", "determiner": "DET", "interjection": "INTJ", 
               "noun": "NOUN", "numeral": "NUM", "particle": "PART", "pronoun": "PRON", 
               "proper noun": "PROPN", "punc": "PUNCT", "subordinating conjunction": "SCONJ", 
               "symbol": "SYM", "verb": "VERB", "other": "X"}

mled = MLEDisambiguator.pretrained()
tagger = DefaultTagger(mled, 'pos')
print(tagger.feature_list())

def get_ar_pos(tokens):
    pos = []
    for i in range(0, len(tokens)):
        t_pos = tagger.tag(tokens[i])
        if len(t_pos) != len(tokens[i]):
            print("mismatch in length")
        pos.append(t_pos)
    return pos

def get_en_pos(tokens):
    pos = []
    for i in range(0, len(tokens)):
        t_pos = []
        for e in nltk.pos_tag(tokens[i]):
            for p in e[1].split("\n"):
                t_pos.append(p)
        if len(t_pos) != len(tokens[i]):
            print("mismatch in length")
        pos.append(t_pos)
    return pos

FileNotFoundError: [Errno 2] No such file or directory: '/home/conite/.camel_tools/data/morphology_db/calima-msa-r13/morphology.db'

In [ ]:
def conver2crf(tokens, pos, labels, out_path):
    with open(out_path, mode='w', encoding="utf-8", newline="") as data_file:
        writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["Sentence #","Word","POS","Tag"])
        for tweet in range(0, len(tokens)):
            begin = True
            for j in range(0, len(tokens[tweet])):
                if begin:
                    writer.writerow(["Sentence: " + str(tweet+1), tokens[tweet][j], pos[tweet][j], labels[tweet][j]])
                    begin = False
                else:
                    writer.writerow(["", tokens[tweet][j], pos[tweet][j], labels[tweet][j]])

In [ ]:
def convert2typeless(labels):
    tllabels = []
    
    for i in range(len(labels)):
        tllabels.append([])
        for l in labels[i]:
            spls = l.split("-")
            if len(spls) > 1:
                tllabels[i].append(spls[0]+"-LOC")
            else:
                tllabels[i].append(l)
    
    return tllabels 

In [ ]:
path = "/home/conite/Documents/WORKSPACE/PROJECTS/Location Mention Recognition (LMR)" + "IDRISI\\data\\LMR\\"
events = ["beirut_explosion_2020", "cairo_bombing_2019", "covid_2019", "dragon_storms_2020",
          "hafr_albatin_floods_2019", "jordan_floods_2018", "kuwait_floods_2018"]

for typ in ['typefull', 'typeless']:
    for case in ['random', 'timebased']:
        for event in events:
            in_path = path + "AR\gold-" + case + "-bilou\\" + event 
            out_path = path + "AR\gold-" + case + "-bilou-crf\\" + typ + "\\" + event 
            if not os.path.exists(out_path):
                os.makedirs(out_path)
            for prt in ["train", "dev", "test"]:
                prt_in_path = in_path + "\\" + prt + ".txt"
                tokens, labels = read_tweets(prt_in_path)
                pos = get_ar_pos(tokens)
                if typ == 'typeless':
                    labels = convert2typeless(labels)
                prt_out_path = out_path + "\\" + prt + ".csv"
                conver2crf(tokens, pos, labels, prt_out_path)


In [ ]:
path = "<path to IDRISI data directory>" + "IDRISI\\data\\LMR\\"
events = ["california_wildfires_2018", "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016", 
          "greece_wildfires_2018", "hurricane_dorian_2019", "hurricane_florence_2018", "hurricane_harvey_2017", 
          "hurricane_irma_2017", "hurricane_maria_2017", "hurricane_matthew_2016", "italy_earthquake_aug_2016", 
          "kaikoura_earthquake_2016", "kerala_floods_2018", "maryland_floods_2018", "midwestern_us_floods_2019", 
          "pakistan_earthquake_2019", "puebla_mexico_earthquake_2017", "srilanka_floods_2017"]

for typ in ['typefull', 'typeless']:
    for case in ['random', 'timebased']:
        for event in events:

            in_path = path + "EN\gold-" + case + "-bilou\\" + event 
            out_path = path + "EN\gold-" + case + "-bilou-crf\\" + typ + "\\" + event 
            if not os.path.exists(out_path):
                os.makedirs(out_path)
            for prt in ["train", "dev", "test"]:
                prt_in_path = in_path + "\\" + prt + ".txt"
                tokens, labels = read_tweets(prt_in_path)
                pos = get_en_pos(tokens)
                if typ == 'typeless':
                    labels = convert2typeless(labels)
                prt_out_path = out_path + "\\" + prt + ".csv"
                conver2crf(tokens, pos, labels, prt_out_path)